## 13주차 과제 order 프로그램 작성 및 배포 사용

In [9]:

%%writefile src/Ex13.sol
pragma solidity ^0.5.10;


contract OrderEvent {
    address owner;
    uint balance;
    constructor() public {
        owner = msg.sender;
        balance = 0;
    }
    uint unitPrice = 1;
    event OrderLog(string);
    event OrderLog(bytes2 _itemId, uint _value);
    event OrderLog(uint256 timestamp);
    event OrderLog(address indexed _from, bytes2 _itemId, uint indexed _value,string _to);

    function order(bytes2 _itemId, uint quantity,string memory _to) public payable {
        uint256 orderTime = now;
        uint256 orderAmount = quantity * unitPrice;
        require(msg.value == orderAmount);
        emit OrderLog("Ordered");
        emit OrderLog(orderTime);
        emit OrderLog(msg.sender, _itemId, msg.value, _to );
    }
    function getBalance() public isOwner payable returns(uint) {
        return(address(this).balance);
    }
    modifier isOwner() {
        require(msg.sender == owner);
        _;
    }
}

Overwriting src/Ex13.sol


In [10]:
!solc --abi --bin --gas src/Ex13.sol


======= src/Ex13.sol:OrderEvent =======
Gas estimation:
construction:
   45518 + 187000 = 232518
external:
   getBalance():	842
   order(bytes2,uint256,string):	infinite
Binary: 
6080604052600160025534801561001557600080fd5b50336000806101000a81548173ffffffffffffffffffffffffffffffffffffffff021916908373ffffffffffffffffffffffffffffffffffffffff16021790555060006001819055506103a78061006d6000396000f3fe6080604052600436106100295760003560e01c806312065fe01461002e5780637a59a9291461004c575b600080fd5b61003661013c565b6040518082815260200191505060405180910390f35b61013a6004803603606081101561006257600080fd5b8101908080357dffffffffffffffffffffffffffffffffffffffffffffffffffffffffffff1916906020019092919080359060200190929190803590602001906401000000008111156100b457600080fd5b8201836020820111156100c657600080fd5b803590602001918460018302840111640100000000831117156100e857600080fd5b91908080601f016020809104026020016040519081016040528093929190818152602001838380828437600081840152601f19601f820116905080830192505050505050

In [11]:

%%writefile src/Ex13Deploy.js
var Web3=require('web3');
var web3 = new Web3(new Web3.providers.HttpProvider("http://localhost:8345"));
var _abiArray =[{"constant":false,"inputs":[],"name":"getBalance","outputs":[{"name":"","type":"uint256"}],"payable":true,"stateMutability":"payable","type":"function"},{"constant":false,"inputs":[{"name":"_itemId","type":"bytes2"},{"name":"quantity","type":"uint256"},{"name":"_to","type":"string"}],"name":"order","outputs":[],"payable":true,"stateMutability":"payable","type":"function"},{"inputs":[],"payable":false,"stateMutability":"nonpayable","type":"constructor"},{"anonymous":false,"inputs":[{"indexed":false,"name":"","type":"string"}],"name":"OrderLog","type":"event"},{"anonymous":false,"inputs":[{"indexed":false,"name":"_itemId","type":"bytes2"},{"indexed":false,"name":"_value","type":"uint256"}],"name":"OrderLog","type":"event"},{"anonymous":false,"inputs":[{"indexed":false,"name":"timestamp","type":"uint256"}],"name":"OrderLog","type":"event"},{"anonymous":false,"inputs":[{"indexed":true,"name":"_from","type":"address"},{"indexed":false,"name":"_itemId","type":"bytes2"},{"indexed":true,"name":"_value","type":"uint256"},{"indexed":false,"name":"_to","type":"string"}],"name":"OrderLog","type":"event"}];
//var _abiArray = JSON.parse(_abiStr);
var _bin="0x" + "6080604052600160025534801561001557600080fd5b50336000806101000a81548173ffffffffffffffffffffffffffffffffffffffff021916908373ffffffffffffffffffffffffffffffffffffffff16021790555060006001819055506103a78061006d6000396000f3fe6080604052600436106100295760003560e01c806312065fe01461002e5780637a59a9291461004c575b600080fd5b61003661013c565b6040518082815260200191505060405180910390f35b61013a6004803603606081101561006257600080fd5b8101908080357dffffffffffffffffffffffffffffffffffffffffffffffffffffffffffff1916906020019092919080359060200190929190803590602001906401000000008111156100b457600080fd5b8201836020820111156100c657600080fd5b803590602001918460018302840111640100000000831117156100e857600080fd5b91908080601f016020809104026020016040519081016040528093929190818152602001838380828437600081840152601f19601f8201169050808301925050505050505091929192905050506101b4565b005b60008060009054906101000a900473ffffffffffffffffffffffffffffffffffffffff1673ffffffffffffffffffffffffffffffffffffffff163373ffffffffffffffffffffffffffffffffffffffff161461019757600080fd5b3073ffffffffffffffffffffffffffffffffffffffff1631905090565b60004290506000600254840290508034146101ce57600080fd5b7fb9e815be6bf0b588396af8fa0244c9f92e26ad4dbf3694c09d90b0cc9f2be8e56040518080602001828103825260078152602001807f4f7264657265640000000000000000000000000000000000000000000000000081525060200191505060405180910390a17f2fb1e4e17675a393b0a43253759334f7e61806d1ac2690f0432362ed1c4c1611826040518082815260200191505060405180910390a1343373ffffffffffffffffffffffffffffffffffffffff167f0cee52e22e6b387c4183ee502522fa33f030e118cc6065c7a842e8b7c00e4a6a878660405180837dffffffffffffffffffffffffffffffffffffffffffffffffffffffffffff19167dffffffffffffffffffffffffffffffffffffffffffffffffffffffffffff1916815260200180602001828103825283818151815260200191508051906020019080838360005b83811015610330578082015181840152602081019050610315565b50505050905090810190601f16801561035d5780820380516001836020036101000a031916815260200191505b50935050505060405180910390a3505050505056fea265627a7a72305820c2aea73e7dba841f1245e025cce33ffc55d63015a454065a6758c55fa0e265a064736f6c634300050a0032";

//unlock the account with a password provided
//web3.personal.unlockAccount(web3.eth.accounts[0],'password');
async function deploy() {
    const accounts = await web3.eth.getAccounts();
    console.log("Deploying the contract from " + accounts[0]);
    var deployed = await new web3.eth.Contract(_abiArray)
        .deploy({data: _bin})
        .send({from: accounts[0], gas: 300000}, function(err, transactionHash) {
                if(!err) console.log("hash: " + transactionHash); 
        })
        //.then(function(newContractInstance) {
        //    console.log(newContractInstance.options.address)
        //});
    console.log("---> The contract deployed to: " + deployed.options.address)
}

deploy()

Overwriting src/Ex13Deploy.js


In [12]:
!node src/Ex13Deploy.js

Deploying the contract from 0x6c5B24a1Da63b1cCa7DbCf8584149207D101Ef61
hash: 0x33d28c412a21549db534ca035486d517d3fbfb65b31dcbdaff97d57bea9cf825
---> The contract deployed to: 0xbc4cf7Aa56aD13448C760CCA462486C922e62cC8


In [13]:
!geth --exec "eth.getTransactionReceipt('0x33d28c412a21549db534ca035486d517d3fbfb65b31dcbdaff97d57bea9cf825')" attach http://localhost:8345


{
  blockHash: "0xe8e33ae1081f991851a6118dbf97ccfcebdbb175f053e4207fbb16855dcb29d2",
  blockNumber: 1,
  contractAddress: "0xbc4cf7aa56ad13448c760cca462486c922e62cc8",
  cumulativeGasUsed: 297806,
  from: "0x6c5b24a1da63b1cca7dbcf8584149207d101ef61",
  gasUsed: 297806,
  logs: [],
  logsBloom: "0x00000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000",
  status: "0x1",
  to: null,
  transactionHash: "0x33d28c412a21549db534ca035486d517d3fbfb65b31dcbdaff97d57bea9cf825",
  transactionIndex: 0
}


In [14]:
%%writefile src/Ex13Use.js
var Web3=require('web3');
var web3;
var fs=require('fs');
if (typeof web3 !== 'undefined') {
    web3 = new Web3(web3.currentProvider);
} else {
    var web3 = new Web3(new Web3.providers.WebsocketProvider("http://localhost:8345"));
}
var _abiArray =[{"constant":false,"inputs":[],"name":"getBalance","outputs":[{"name":"","type":"uint256"}],"payable":true,"stateMutability":"payable","type":"function"},{"constant":false,"inputs":[{"name":"_itemId","type":"bytes2"},{"name":"quantity","type":"uint256"},{"name":"_to","type":"string"}],"name":"order","outputs":[],"payable":true,"stateMutability":"payable","type":"function"},{"inputs":[],"payable":false,"stateMutability":"nonpayable","type":"constructor"},{"anonymous":false,"inputs":[{"indexed":false,"name":"","type":"string"}],"name":"OrderLog","type":"event"},{"anonymous":false,"inputs":[{"indexed":false,"name":"_itemId","type":"bytes2"},{"indexed":false,"name":"_value","type":"uint256"}],"name":"OrderLog","type":"event"},{"anonymous":false,"inputs":[{"indexed":false,"name":"timestamp","type":"uint256"}],"name":"OrderLog","type":"event"},{"anonymous":false,"inputs":[{"indexed":true,"name":"_from","type":"address"},{"indexed":false,"name":"_itemId","type":"bytes2"},{"indexed":true,"name":"_value","type":"uint256"},{"indexed":false,"name":"_to","type":"string"}],"name":"OrderLog","type":"event"}];
//var _abiArray=JSON.parse(_abiStr);

async function doIt() {
    var _order = new web3.eth.Contract(_abiArray, '0xbc4cf7Aa56aD13448C760CCA462486C922e62cC8');
    const accounts = await web3.eth.getAccounts();
    console.log("Account: " + accounts[0]);
    var event = _order.events.OrderLog({
            //filter: {_from: accounts[0], _value: [20,50]},
            filter: {_from: accounts[0], _value: [3,5,20]},
            fromBlock: 'latest', toBlock: 'pending'
        }, function (error, result) {
        if (!error) {
            log =JSON.stringify(result.returnValues);
            //console.log("Event fired: " + JSON.stringify(result) + "\n---> " + JSON.stringify(result.returnValues));
            console.log("Event fired: " + JSON.stringify(result.returnValues) + "\n---> ");
          //fs.writeFile("src/OrderEvent.txt","\n", log, "utf-8", function(e) {
            fs.appendFile("src/OrderEvent.txt",log+"\n", "utf-8", function(e) {
                if(!e) {
                    console.log(">> Writing to file");
                }
            });
        }
    });
    var value;
    const balanceBefore =  await web3.eth.getBalance(accounts[0]);
    console.log("Balance before: " + balanceBefore);
    // this will fire an event
    my =  await _order.methods.order("0x1111", 3, "20 2-gil Hongji-dong Jongro-gu Seoul")
        .send({from: accounts[0], gas: 100000, value:3});
        //.then(function(my) {console.log("---> MyFunction called " + JSON.stringify(my.events.OrderLog.returnValues));});
    console.log("---> MyFunction called " + JSON.stringify(my.events.OrderLog.returnValues));
    // this will fire another event
    my =await _order.methods.order("0x1111", 5,"20 2-gil Hongji-dong Jongro-gu Seoul" ).send({from: accounts[0], gas: 100000, value:5});
    console.log("---> MyFunction called " + JSON.stringify(my.events.OrderLog.returnValues));
    // this will NOT fire another event
    my =await _order.methods.order("0x1111", 20,"20 2-gil Hongji-dong Jongro-gu Seoul").send({from: accounts[0], gas: 100000, value:20});
    console.log("---> MyFunction called " + JSON.stringify(my.events.OrderLog.returnValues));
    const balanceAfter = await web3.eth.getBalance(accounts[0]);
    console.log("Balance after: " + balanceAfter);
    console.log("Balance diff: " + (balanceBefore - balanceAfter));
}

doIt()

Overwriting src/Ex13Use.js


In [15]:
!node src/Ex13Use.js

Account: 0x6c5B24a1Da63b1cCa7DbCf8584149207D101Ef61
Balance before: 99994043880000000000
Event fired: {"0":"0x6c5B24a1Da63b1cCa7DbCf8584149207D101Ef61","1":"0x1111","2":"3","3":"20 2-gil Hongji-dong Jongro-gu Seoul","_from":"0x6c5B24a1Da63b1cCa7DbCf8584149207D101Ef61","_itemId":"0x1111","_value":"3","_to":"20 2-gil Hongji-dong Jongro-gu Seoul"}
---> 
>> Writing to file
---> MyFunction called undefined
Event fired: {"0":"0x6c5B24a1Da63b1cCa7DbCf8584149207D101Ef61","1":"0x1111","2":"5","3":"20 2-gil Hongji-dong Jongro-gu Seoul","_from":"0x6c5B24a1Da63b1cCa7DbCf8584149207D101Ef61","_itemId":"0x1111","_value":"5","_to":"20 2-gil Hongji-dong Jongro-gu Seoul"}
---> 
>> Writing to file
---> MyFunction called undefined
Event fired: {"0":"0x6c5B24a1Da63b1cCa7DbCf8584149207D101Ef61","1":"0x1111","2":"20","3":"20 2-gil Hongji-dong Jongro-gu Seoul","_from":"0x6c5B24a1Da63b1cCa7DbCf8584149207D101Ef61","_itemId":"0x1111","_value":"20","_to":"20 2-gil Hongji-dong Jongro-gu Seoul"}
---> 
>> Writing to

## type 명령어 결과 출력

In [16]:
!type src\OrderEvent.txt

{"0":"0x6c5B24a1Da63b1cCa7DbCf8584149207D101Ef61","1":"0x1111","2":"3","3":"20 2-gil Hongji-dong Jongro-gu Seoul","_from":"0x6c5B24a1Da63b1cCa7DbCf8584149207D101Ef61","_itemId":"0x1111","_value":"3","_to":"20 2-gil Hongji-dong Jongro-gu Seoul"}
{"0":"0x6c5B24a1Da63b1cCa7DbCf8584149207D101Ef61","1":"0x1111","2":"5","3":"20 2-gil Hongji-dong Jongro-gu Seoul","_from":"0x6c5B24a1Da63b1cCa7DbCf8584149207D101Ef61","_itemId":"0x1111","_value":"5","_to":"20 2-gil Hongji-dong Jongro-gu Seoul"}
{"0":"0x6c5B24a1Da63b1cCa7DbCf8584149207D101Ef61","1":"0x1111","2":"20","3":"20 2-gil Hongji-dong Jongro-gu Seoul","_from":"0x6c5B24a1Da63b1cCa7DbCf8584149207D101Ef61","_itemId":"0x1111","_value":"20","_to":"20 2-gil Hongji-dong Jongro-gu Seoul"}
